In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%matplotlib inline
from copy import copy
import gc
import joblib
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import sys
from warnings import simplefilter

import tensorflow as tf
from tensorflow.keras import Model, Input, backend as K
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Dropout
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split



In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
def bertfortest(data): # train set과 test set을 전처리 해준다.
    # [CLS] + 문장 + [SEP]
    sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in data['sentence']]

    # 라벨 데이터
    labels = data['label']

    # Word 토크나이저 토큰화
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False,padding="max_length")
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 시퀀스 설정 및 정수 인덱스 변환 & 패딩
    MAX_LEN = 128
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

    # 패딩 및 잘라내기
    for i, ids in enumerate(input_ids):
        if len(ids) < MAX_LEN:
            input_ids[i] = ids + [tokenizer.pad_token_id] * (MAX_LEN - len(ids))
        else:
            input_ids[i] = ids[:MAX_LEN]
    # 어텐션 마스크
    attention_masks = []
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 파이토치 텐서로 변환
    inputs = torch.tensor(input_ids)
    labels = torch.tensor(labels)
    masks = torch.tensor(attention_masks)

    dataset = TensorDataset(inputs, masks, labels)
    sampler = SequentialSampler(dataset)
    dataloader = DataLoader(dataset, sampler=sampler, batch_size=100)

    return dataloader

In [ ]:
# # 정확도 계산 함수
# def flat_accuracy(preds, labels):

#     pred_flat = np.argmax(preds, axis=1).flatten()
#     labels_flat = labels.flatten()


#     return np.sum(pred_flat == labels_flat) / len(labels_flat)

def flat_accuracy_with_mismatched_indices(preds, labels, index):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    accuracy = np.sum(pred_flat == labels_flat) / len(labels_flat)

    print(pred_flat)
    print(labels_flat)
    return accuracy


# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))

    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
pip install datasets

In [ ]:
model=BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/인지개/model_epoch2folds3_1.pt')
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
from datasets import load_dataset
import time
import datetime
from torch.utils.data.sampler import SubsetRandomSampler
from keras.models import load_model

t0 = time.time()

df = pd.read_csv("/content/drive/MyDrive/인지개/test_dataset.csv")

test_ds = load_dataset("csv", data_files="/content/drive/MyDrive/인지개/test_dataset.csv")['train']
X_test, y_test = test_ds['sentence'], test_ds['label']

labels = df['label'].values

test_ds = load_dataset("csv", data_files="/content/drive/MyDrive/인지개/final_csv/responses9.csv")['train']
test_dataloader=bertfortest(test_ds)

model.eval()

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

pred_result=[]

for batch in test_dataloader:

  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch

  with torch.no_grad():
    outputs = model(b_input_ids,token_type_ids=None,attention_mask=b_input_mask)
  logits = outputs[0]

  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  pred_flat_REB = np.argmax(logits, axis=1).flatten()



In [ ]:
from sklearn.metrics import accuracy_score, classification_report

print(classification_report(pred_flat_REB,y_test))

              precision    recall  f1-score   support

           0       0.98      0.95      0.96        56
           1       0.93      0.98      0.96        44

    accuracy                           0.96       100
   macro avg       0.96      0.96      0.96       100
weighted avg       0.96      0.96      0.96       100



In [ ]:
from sklearn.metrics import accuracy_score, classification_report

print(classification_report(pred_flat_basic,y_test))

In [ ]:

t0 = time.time()

test_ds = load_dataset("csv", data_files="/content/drive/MyDrive/인지개/test_dataset.csv")['train']
test_dataloader=bertfortest(test_ds)

model.eval()

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
      outputs = model(b_input_ids,token_type_ids=None,attention_mask=b_input_mask)
    logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    pred_flat_basic = np.argmax(logits, axis=1).flatten()

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

print(classification_report(pred_flat_basic,y_test))

              precision    recall  f1-score   support

           0       0.96      0.93      0.95        56
           1       0.91      0.95      0.93        44

    accuracy                           0.94       100
   macro avg       0.94      0.94      0.94       100
weighted avg       0.94      0.94      0.94       100

